In [1]:
from glob import glob
import igraph
import numpy as np
from tqdm import tqdm

import g2s
from g2s.graph_extractor import xyz2mol_graph
from g2s.representations.locality import get_local_environment, get_local_knn_environment

RDKit WARNING: [09:17:20] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
ci_path = sorted(glob('/Users/c0uch1/work/g2g_data/qm9/H10C7O2/*.xyz'))

In [3]:
def prep_data(filepath):
    bond_order_matrix, nuclear_charges, coords = xyz2mol_graph(filepath)

    distances = g2s.utils.calculate_distances(coords)
    gc = g2s.GraphCompound(bond_order_matrix, nuclear_charges, distances)
    gc.generate_bond_length(size=19, sorting='norm_row')

    representation = gc.representation
    adjacency = gc.adjacency_matrix
    nuclear_charges = gc.nuclear_charges
    distances = gc.distances

    representation = g2s.utils.vector_to_square([representation])[0]
    distances = g2s.utils.vector_to_square([distances])[0]

    hydrogen_idxs = np.where(nuclear_charges == 1)[0]
    heavy_atom_idxs = np.where(nuclear_charges != 1)[0]

    ha_representation = representation[heavy_atom_idxs][:, heavy_atom_idxs]
    hy_representation = representation[hydrogen_idxs][:, heavy_atom_idxs]

    ha_dm = distances[heavy_atom_idxs][:, heavy_atom_idxs]
    ha_nc = nuclear_charges[heavy_atom_idxs]
    
    ha_adj = adjacency[heavy_atom_idxs][:, heavy_atom_idxs]

    ha_local_repr = []
    ha_local_nc = []
    ha_local_dist = []
    ha_local_idxs = []

    hy_local_repr = []
    hy_local_nc = []
    hy_local_dist = []
    hy_local_idxs = []

    for i in range(len(heavy_atom_idxs)):
        atomic_representation, local_nuclear_charges, local_idxs, local_distances = get_local_knn_environment(i,  ha_adj, ha_representation, ha_nc, distances=ha_dm, depth=2)

        ha_local_repr.append(atomic_representation)
        ha_local_dist.append(local_distances)
        ha_local_nc.append(local_nuclear_charges)
        ha_local_idxs.append(local_idxs)

    for i in hydrogen_idxs:
        hydro_atomic_representation, hydro_local_nuclear_charges, hydro_local_idxs, hydro_local_distances = get_local_environment(i, representation, nuclear_charges, distances, n_neighs=4)
        hy_local_repr.append(hydro_atomic_representation)
        hy_local_nc.append(hydro_local_nuclear_charges)
        hy_local_dist.append(hydro_local_distances)
        hy_local_idxs.append(hydro_local_idxs)
        
    mol_data = {'ha_lr': ha_local_repr,
                'ha_lnc': ha_local_nc,
                'ha_ldist': ha_local_dist,
                'ha_lidxs': ha_local_idxs,
                'hy_lr': hy_local_repr,
                'hy_lnc': hy_local_nc,
                'hy_ldist': hy_local_dist,
                'hy_lidxs': hy_local_idxs,
                'nuclear_charges': nuclear_charges,
                'heavy_atom_idxs': heavy_atom_idxs,
                'hydrogen_idxs': hydrogen_idxs,
                'full_dm': distances}
   
    return mol_data

In [4]:
mol_envs = []

for ci in tqdm(ci_path):
    mol_envs.append(prep_data(ci))

100%|██████████| 6095/6095 [00:37<00:00, 162.09it/s]


In [5]:
ha_lr = [mol_envs[i]['ha_lr'] for i in range(len(mol_envs))]
hy_lr = [mol_envs[i]['hy_lr'] for i in range(len(mol_envs))]
ha_lidxs = [mol_envs[i]['ha_lidxs'] for i in range(len(mol_envs))]
hy_lidxs = [mol_envs[i]['hy_lidxs'] for i in range(len(mol_envs))]
ha_ldist = [mol_envs[i]['ha_ldist'] for i in range(len(mol_envs))]
hy_ldist = [mol_envs[i]['hy_ldist'] for i in range(len(mol_envs))]

In [6]:
uq_ha_env = g2s.representations.locality.get_unique_environments(ha_lr, ha_lidxs, local_distances=ha_ldist)
uq_hy_env = g2s.representations.locality.get_unique_environments(hy_lr, hy_lidxs, local_distances=hy_ldist)

In [311]:
training_ha_envs = []
training_ha_labels = []

training_hy_envs = []
training_hy_labels = []

for k in uq_ha_env.keys():
    training_ha_envs.append(k)
    training_ha_labels.append(uq_ha_env[k]['avg_dist'])
    

for k in uq_hy_env.keys():
    training_hy_envs.append(k)
    training_hy_labels.append(uq_hy_env[k]['avg_dist'])
    
training_ha_envs = np.array(training_ha_envs)
training_ha_labels = np.array(training_ha_labels)
training_hy_envs = np.array(training_hy_envs)
training_hy_labels = np.array(training_hy_labels)

/Users/c0uch1/.miniconda3/envs/wk/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()
/Users/c0uch1/.miniconda3/envs/wk/lib/python3.7/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [50]:
kernel_sigma = 32
kernel_lambda = 1e-5

train_kernel = g2s.krr.laplacian_kernel(training_ha_envs, training_ha_envs, 32)
train_kernel[np.diag_indices_from(train_kernel)] += 1e-5
alphas = g2s.krr.train_multikernel(train_kernel, training_ha_labels)

pred_distances = g2s.krr.predict_distances(train_kernel, alphas)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [7]:
sparse_distances = []
mol_nuclear_charges =[]
for i in tqdm(range(len(mol_envs))):
    sp, mnc = g2s.representations.locality.get_sparse_bounds_matrix(uq_ha_env, mol_envs[i], boundaries=True, tight_envs=False)
    sparse_distances.append(sp)
    mol_nuclear_charges.append(mnc)

100%|██████████| 6095/6095 [00:05<00:00, 1050.90it/s]


In [47]:
dgsol = DGSOL(np.array(sparse_distances),  np.array(mol_nuclear_charges), vectorized_input=False)
dgsol.solve_distance_geometry('/Users/c0uch1/work/test/')
outdir = sorted(glob('/Users/c0uch1/work/test/*'))
n_confs = 5
for i, e in enumerate(outdir):
    for j in range(n_confs):
        g2s.utils.write_xyz(f'{e}/dgsol_conf_{j}.xyz', dgsol.coords[i][j], dgsol.nuclear_charges[i])

100%|██████████| 6095/6095 [06:37<00:00, 15.34it/s]


In [350]:
# potential_bases = [[i, *np.where(sparse_dm[i] != 0.)[0]] for i in range(len(sparse_dm)) if np.where(sparse_dm[i] != 0.)[0].shape[0] >= 4]
# lstB = np.array([1, 0, 2, 4])
# setB = np.array([False]*9)
# setB[lstB] = True

In [8]:
potential_bases = [[i, *np.where(sparse_distances[100][i] != 0.)[0]] for i in range(len(sparse_distances[100])) if np.where(sparse_distances[100][i] != 0.)[0].shape[0] >= 4]

In [22]:
bases = [sorted([i, *np.random.choice(potential_bases[i][1:], 3, replace=False)]) for i in range(9)]
setB = np.tile(np.array([False]*9),9).reshape(9,9)

for i in range(9):
    setB[i][bases[i]] = True

In [13]:
def sample_random(outdir, sparse_distances, nuclear_charges, n_confs, seed=1337):
    for i in tqdm(range(len(sparse_distances))):
        np.random.seed(seed)

        n_atoms = len(nuclear_charges[i])
        pb = [[k, *np.where(sparse_distances[i][k] != 0.)[0]] for k in range(n_atoms) if np.where(sparse_distances[i][k] != 0.)[0].shape[0] >= 4]
        coords = []
        for j in range(n_confs):
            k = np.random.randint(0, len(pb))
            t = np.random.uniform(0.1, 1.0)
            base = sorted([k, *np.random.choice(pb[k][1:], 3, replace=False)])
            setB = np.array([False] * n_atoms)
            setB[base] = True
            coords.append(lsbuild(sparse_distances[i], nuclear_charges[i], lstB=base, setB=setB, t=t)['x'])

In [30]:
sample_random('', sparse_distances, mol_nuclear_charges, 10)





  0%|          | 0/6095 [00:00<?, ?it/s]

Base (maxRelRes: 0.000e+00)
[[ 1.49572195  0.11451524  0.08920142]
 [ 3.67122621  0.63776084 -0.02694085]
 [ 3.52815134 -0.71217106 -0.00978262]]
[ 0.86187028 82.15089007 82.61600578]
[  21.10452371   -6.84480574 -335.42958057]
[[-1.138023   -0.59680512 -0.7897741 ]
 [-2.48288424 -0.19726053 -0.26239678]
 [-4.55850331  0.00595761  0.55380581]
 [-3.82163136  1.08902231  0.9094023 ]]
[50.42460255 14.44390857 39.00508507 16.04349573]
[-10.83505806  -5.46318684 -25.92587622]
[[-1.25572704 -0.40922656 -0.72849285]
 [-1.36880018  0.14601625 -2.12201672]
 [-2.49496023 -0.19971349  0.08812168]
 [-4.52689913 -0.10510199  1.02511396]
 [-3.81824976  1.00238198  1.36238761]]
[ 42.47080661 127.17522172  -8.8590446  -69.05331641 -88.92584831]
[  1.47380262  -0.30130616 -60.88739451]
[[-1.21856751 -0.35577319 -0.81560059]
 [-1.23548553  0.25124121 -2.19278866]
 [-2.49929048 -0.14663639 -0.06746588]
 [-3.45482983 -1.00800166  0.37214357]
 [-4.38105567 -0.22809589  1.14340786]
 [-3.89640365  1.04007578





  0%|          | 1/6095 [00:03<5:45:51,  3.41s/it]

Base (maxRelRes: 8.957e-02)
   NLP   (maxRelRes: 0.000e+00)
[[ 1.46649368 -0.32128713  0.30985195]
 [ 3.66608177  0.04367139  0.51073158]
 [ 2.5127005   0.65982907  0.15376986]]
[ 1.09366409 81.83737316  1.82250589]
[-1028145.34767145  2239988.57069513  7188753.28553453]
[[ 0.85196937 -0.70065526 -0.90677294]
 [ 2.30505369 -0.87998862 -0.45267277]
 [ 4.46099871 -0.35230565 -0.16080431]
 [ 3.30769802  0.13138861 -0.68358314]]
[-1.74614531 -0.81294081 80.36901799  0.32819759]
[ 14.36666026 -13.68715494  46.94965998]
[[ 6.48596837e-01 -2.40872606e-01 -2.35342338e+00]
 [ 7.87698590e-01 -7.49799459e-01 -9.24819837e-01]
 [ 2.21470176e+00 -1.09338764e+00 -4.89124346e-01]
 [ 4.28458891e+00 -7.45613065e-01  2.89559075e-01]
 [ 3.13157405e+00 -9.35426921e-02  1.94818412e-03]]
[-17.31781729  -5.32823359   3.08278845  94.52788067   8.18177773]
[12.98055013 -1.62142663 15.07691547]
[[ 0.8119587   1.57391334 -1.60481527]
 [ 0.88804888  1.12961475 -0.13328996]
 [ 0.32094103  2.22012457  0.82819793]
 [





  0%|          | 2/6095 [00:05<4:57:58,  2.93s/it]

...





  3%|▎         | 170/6095 [05:13<2:37:03,  1.59s/it]

[[ 2.95234969  1.05367442 -0.03499636]
 [-0.71840388  1.89132422  0.69313897]
 [-0.85143082  1.03557682 -0.61826029]
 [ 0.66680875  1.20417896  0.5354947 ]
 [ 2.12584252  1.47567431  1.07069844]
 [ 2.80557384  1.95972899 -1.14287981]
 [ 0.45064092  1.85340584 -0.87125843]]
[ -93.4726211   -31.60826988 -142.92917722 -127.39309373   10.68865052
 -102.19586727 -133.17059762]
[  4.65050854 -53.88398072  45.14960802]
Base (maxRelRes: 3.333e+00)
   NLP   (maxRelRes: 0.000e+00)
[[ 2.71456988  1.65714578 -0.7120014 ]
 [ 4.09704313  2.33784812 -0.52382853]
 [ 0.31787533  0.68738042 -1.22954553]]
[-14.55210263 -26.98621448  -2.83774696]
[ 32.42907554 -75.42727966 -31.47592677]
[[-2.28440825 -2.08547625  0.78770888]
 [ 0.63746683 -1.96828002 -0.86379589]
 [ 1.26338731 -0.63760605 -0.35740748]
 [-2.1685675  -1.58059278 -0.56023511]]
[22.09977762 34.99349124 14.70629299 21.50924665]
[  2.35538307 -15.01396071  -4.87173715]
[[-2.39895759 -2.11602378  0.7656084 ]
 [ 0.59264501 -1.9016742  -0.96172364





  3%|▎         | 171/6095 [05:14<2:24:58,  1.47s/it]

[[ 0.09899686 -1.87012097 -2.6642703 ]
 [-0.18798929  0.02379942 -3.40013373]
 [ 1.16169168 -0.80762533 -2.99534843]
 [ 2.16954626 -0.3023207  -1.82013002]
 [ 1.41114432  0.05718756 -0.39747091]
 [-1.10282344 -0.42150081 -0.89107771]
 [-1.1076073  -0.92535927 -2.46047943]]
[-270.56698203 -169.99939165 -193.38859824 -100.35862931   -2.0937735
  -98.53697371 -225.95387064]
[13.00023766 75.47362619 51.74319472]
Base (maxRelRes: 0.000e+00)
[[ 2.23468272 -1.32271487  0.06092517]
 [ 2.37094396  1.24368717  0.06558936]
 [ 1.45280154  0.00491154 -0.20075495]]
[83.37088157 -5.24696298 -0.42604711]
[ 13.6139225  -35.43414591  99.7750641 ]
[[-1.13746531 -1.49927681  1.33114093]
 [ 0.89949674 -2.82294337  2.13979719]
 [ 1.06393591 -0.57414546  0.7636571 ]
 [ 0.2591787  -1.47865286  1.72908404]]
[ 36.06239377 -24.50576332  22.56487261 -39.23596085]
[-13.21413021  -1.32431606  -4.79352008]
[[-1.08241072 -0.94843709  1.80008372]
 [ 1.43136416 -2.04753494  4.30283886]
 [ 0.93526097 -2.16888293  2.8455

KeyboardInterrupt: 

In [12]:
from g2s.dgeom.lsbuild import lsbuild

# coords = [lsbuild(sparse_distances[100], mol_nuclear_charges[100], lstB=None, setB=None, t=i)['x'] for i in np.arange(0.1, 1.0, 0.1)]
# coords = [lsbuild(sparse_distances[100], mol_nuclear_charges[100], lstB=bases[i], setB=setB[i], t=0.5)['x'] for i in range(9)]
coords = []
for i in tqdm(range(len(sparse_distances))):
    coords.append(lsbuild(sparse_distances[i], mol_nuclear_charges[i]))

  0%|          | 0/6095 [00:00<?, ?it/s]

Updating Bmax (Blen = 9)
Optimal base [2, 3, 4, 5] found after 126 tests.
Base (maxRelRes: 8.779e-17)


  0%|          | 2/6095 [00:02<2:51:11,  1.69s/it]

Updating Bmax (Blen = 9)
Optimal base [0, 1, 2, 3] found after 126 tests.
Base (maxRelRes: 1.712e+00)
   NLP   (maxRelRes: 0.000e+00)
Updating Bmax (Blen = 9)
Optimal base [0, 1, 2, 3] found after 126 tests.
Base (maxRelRes: 2.773e+00)
   NLP   (maxRelRes: 0.000e+00)


  0%|          | 5/6095 [00:02<1:29:55,  1.13it/s]

...

  0%|          | 8/6095 [00:03<39:28,  2.57it/s]  

Updating Bmax (Blen = 9)
Optimal base [0, 1, 2, 3] found after 126 tests.
Base (maxRelRes: 0.000e+00)
Updating Bmax (Blen = 9)
Optimal base [0, 1, 2, 3] found after 126 tests.
Base (maxRelRes: 0.000e+00)


  0%|          | 9/6095 [00:03<39:21,  2.58it/s]

Updating Bmax (Blen = 9)
Optimal base [0, 1, 2, 3] found after 126 tests.
Base (maxRelRes: 1.702e+00)
   NLP   (maxRelRes: 0.000e+00)


KeyboardInterrupt: 

In [ ]:
for i in range(len(coords)):
    g2s.utils.write_xyz(f'test/lsbuild_{i}.xyz', coords[i]['x'],  mol_nuclear_charges[i])

In [364]:
g2s.utils.write_xyz(f'test/lsbuild.xyz', coords['x'], mol['nuclear_charges'][:9])

In [365]:
import py3Dmol
with open(f'test/lsbuild.xyz', 'r') as infile:
    xyz_pred = infile.readlines()
xyz_pred = ''.join(xyz_pred)
with open(f'/Users/c0uch1/work/g2g_data/qm9/H10C7O2/dsgdb9nsd_5000.xyz', 'r') as infile:
    xyz_ref = infile.readlines()
    xyz_ref = ''.join(xyz_ref)
xyzview = py3Dmol.view(width=400,height=400)
xyzview.addModel(xyz_pred,'xyz')
xyzview.addModel(xyz_ref,'xyz')
xyzview.setStyle({'stick':{'colorscheme':'cyanCarbon'}})
xyzview.zoomTo()
xyzview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [29]:
import numpy as np
import networkx as nx
from numpy import sqrt, zeros, sum
from numpy.linalg import svd, norm, lstsq
from scipy.optimize import minimize
from numba import jit

from g2s.constants import vdw_radii


class edge_t:
    def __init__(self, i, j, l, u):
        self.i = i
        self.j = j
        self.l = l
        self.u = u


class get_graph:
    def __init__(self, dmat, nuclear_charges):
        self.edgeEqu = []  # dij == lij == uij (exact distance information)
        self.edgeBnd = []  # lij <= dij <= uij (bound distance information)

        n, m = np.triu_indices(dmat.shape[1], k=1)
        for i, j in zip(n, m):
            u = dmat[i, j]
            l = dmat[j, i]
            if dmat[i, j] == 0.0:
                u = 20.
                l = 1. if nuclear_charges is None else vdw_radii[nuclear_charges[i]] + vdw_radii[
                    nuclear_charges[j]]
            edge = edge_t(i, j, l, u)
            if l == u:
                self.edgeEqu.append(edge)
            else:
                self.edgeBnd.append(edge)
        self.G = self.init_graph()
        self.numNodes = len(self.G)
        self.numEdgeEqu = len(self.edgeEqu)
        self.numEdgeBnd = len(self.edgeBnd)

    def init_graph(self):
        G = nx.Graph()
        for edge in self.edgeEqu + self.edgeBnd:
            G.add_edge(edge.i, edge.j, l=edge.l, u=edge.u)
        return G


def calcNodeMaxRelRes(G, lstB, setB, x, i):
    maxRelRes = 0.0
    if i in lstB:
        xi = x[i]
        for j in G.neighbors(i):
            if not setB[j]:  # only nodes whose coords are fixed
                continue
            xj = x[j]
            dij = norm(xi - xj)
            gij = G[i][j]
            lij, uij = gij['l'], gij['u']
            mij = (lij + uij)
            relResL, relResU = (lij - dij) / mij, (dij - uij) / mij
            # ijMaxRelErr = max(relErrL, relErrU)
            relRes = relResU if relResU > relResL else relResL
            if relRes > 0 and maxRelRes < relRes:  # maxRelErr can be negative
                maxRelRes = relRes
    else:
        for j in G.neighbors(i):
            if not setB[j]:
                continue
            relRes = calcNodeMaxRelRes(G, lstB, setB, x, j)
            if relRes > maxRelRes:
                maxRelRes = relRes
    return maxRelRes


def calcBaseMaxRelRes(G, lstB, setB, x):
    n = len(lstB)
    maxRelRes = 0.0
    for k in range(n):
        i = lstB[k]
        xi = x[i]
        for j in G.neighbors(i):
            if j > i or not setB[j]:
                continue
            xj = x[j]
            dij = norm(xi - xj)
            gij = G[i][j]
            lij, uij = gij['l'], gij['u']
            mij = (lij + uij)
            relResL, relResU = (lij - dij) / mij, (dij - uij) / mij
            # ijMaxRelErr = max(relErrL, relErrU)
            relRes = relResU if relResU > relResL else relResL
            if relRes > 0 and maxRelRes < relRes:  # maxRelErr can be negative
                maxRelRes = relRes
    return maxRelRes


@jit(nopython=True, fastmath=True, cache=True)
def smoothMaxZero(z, tau, lam):
    lam_z = lam * z
    fz = lam_z + sqrt(lam_z ** 2 + tau)
    gz = lam * fz / (fz - lam_z)
    return fz, gz


@jit(nopython=True, fastmath=True, cache=True)
def smoothDistance(yi, yj, tau):
    fy = sqrt(sum(yi ** 2) + sum(yj ** 2) - 2 * (yi @ yj) + tau)
    gz = (yi - yj) / fy
    return fy, gz


@jit(nopython=True, fastmath=True, cache=True)
def funNLP(y, tau, lam, L, U):
    f = 0.0
    g = np.zeros(len(y))
    for l, u in zip(L, U):
        # for i in L:
        i, j, l_val = l
        u_val = u[2]

        yi = y[(3 * i):(3 * (i + 1))]
        # for j in L[i]:
        yj = y[(3 * j):(3 * (j + 1))]
        fdij, gdij = smoothDistance(yi, yj, tau)
        zlij = l_val - fdij
        zuij = fdij - u_val
        fmlij, gmlij = smoothMaxZero(zlij, tau, lam)
        fmuij, gmuij = smoothMaxZero(zuij, tau, lam)
        aij = gmuij - gmlij
        f += fmlij + fmuij
        g[(3 * i):(3 * (i + 1))] += aij * gdij
        g[(3 * j):(3 * (j + 1))] -= aij * gdij
    return (f, g)


def checkDiff(f, y, args, rad=1, ntests=50, tol=1e-3):
    # check derivatives of f around y using numerical approximation as probe.
    ynrm = norm(y)
    dy = ynrm / 1000
    maxRelErr = 0
    for _ in range(ntests):
        x = y + np.random.uniform(-1, 1, size=y.shape) * ynrm
        _, g = f(x, args)
        gnum = np.zeros(y.shape, dtype=float)
        xfw = x.copy()
        xbk = x.copy()
        for i in range(len(y)):
            xfw[i] = x[i] + dy
            xbk[i] = x[i] - dy
            ffw, _ = f(xfw, args)
            fbk, _ = f(xbk, args)
            gnum[i] = (ffw - fbk) / (2 * dy)
            # reset xfw and xbw
            xfw[i] = x[i]
            xbk[i] = x[i]
        relErr = norm(g - gnum) / norm(g)
        if relErr > maxRelErr:
            maxRelErr = relErr
        if relErr > tol:
            raise Exception('Numerical diff and g are different.')


def saveResult(nmr, summary, ans):
    fn_ans = nmr.file.replace('.nmr', '.ans')
    print('Writing ' + fn_ans)
    with open(fn_ans, 'w') as fid:
        fid.write(summary + '\n')
        fid.write('\nCoords [ x y z ] =====\n')
        x = ans['x']
        for i in range(nmr.numNodes):
            fid.write('% 12.8g % 12.8g % 12.8g\n' %
                      (x[i, 0], x[i, 1], x[i, 2]))


def lsbuild(distance_matrix, nuclear_charges, lstB=None, setB=None, t=0.5):
    nmr = get_graph(distance_matrix, nuclear_charges)
    tolMaxRelRes = 1e-4  # tolerance of distance constraints
    G = nmr.G
    if lstB is None and setB is None:
        lstB, setB = initB(G)
    x = initX(G, lstB, t=t)
    maxRelRes = calcBaseMaxRelRes(G, lstB, setB, x)
    print('Base (maxRelRes: %.3e)' % maxRelRes)
    if maxRelRes > tolMaxRelRes:  # refining solution?
        solveNLP(G, setB, lstB, x, False, None)
        maxRelRes = calcBaseMaxRelRes(G, lstB, setB, x)
        print('   NLP   (maxRelRes: %.3e)' % maxRelRes)
    # rank of unsolved (not fixed) atoms
    nodeRnk, nodeIdx = initNodeRnk(G, lstB, setB, x)
    for k in range(len(lstB), nmr.numNodes):
        i = nodeRnk[k]['node']
        if nodeRnk[k]['deg'] < 4:
            # There is not enough info to set i-th node
            print('Warning: Not all nodes were solved.')
            break
        # add i to the solved set of vertices
        setB[i] = True
        lstB.append(i)
        nodeRnk[k]['deg'] = np.inf
        # calc coords
        solveLSTSQ(G, setB, x, i, t=t)
        maxRelRes = calcNodeMaxRelRes(G, lstB, setB, x, i)
        if maxRelRes > tolMaxRelRes:
            solveNLP(G, setB, lstB, x, False, None)
            maxRelRes = calcBaseMaxRelRes(G, lstB, setB, x)
        # TODO This step is O(n * max(deg)) and it could be approximately done
        updtNodeRnk(G, lstB, setB, x, i, nodeRnk, nodeIdx)
    notSolvedNodes = [i for i in range(nmr.numNodes) if not setB[i]]
    for k in notSolvedNodes:
        i = nodeRnk[k]['node']
        if nodeRnk[k]['deg'] < 4:
            # There is not enough info to set i-th node
            print('Warning: Not all nodes were solved.')
            break
        # add i to the solved set of vertices
        setB[i] = True
        lstB.append(i)
        nodeRnk[k]['deg'] = np.inf
        # calc coords
        solveLSTSQ(G, setB, x, i, t=t)
        maxRelRes = calcNodeMaxRelRes(G, lstB, setB, x, i)
        if maxRelRes > tolMaxRelRes:
            solveNLP(G, setB, lstB, x, False, None)
            maxRelRes = calcBaseMaxRelRes(G, lstB, setB, x)
        # TODO This step is O(n * max(deg)) and it could be approximately done
        updtNodeRnk(G, lstB, setB, x, i, nodeRnk, nodeIdx)
    ans = {
        'x': x,
        'maxRelRes': maxRelRes,
        'numSolvedNodes': len(lstB),
        'notSolvedNodes': notSolvedNodes,
    }
    return ans


def updtNodeRnk(G, lstB, setB, x, i, nodeRnk, nodeIdx):
    # update maxRelRes of nodes on B
    for j in lstB:
        jidx = nodeIdx[j]
        nodeRnk[jidx]['maxRelRes'] = calcNodeMaxRelRes(G, lstB, setB, x, j)

    # update naxRelRes of nodes out of B
    V = np.zeros(len(nodeRnk), dtype=bool)  # set of updated vertices
    V[lstB] = True
    for k in lstB:
        for j in G.neighbors(k):
            if V[j]:
                continue
            V[j] = True
            jidx = nodeIdx[j]
            nodeRnk[jidx]['maxRelRes'] = calcNodeMaxRelRes(G, lstB, setB, x, j)

    # update degree of nodes out of B
    for j in G.neighbors(i):
        if j in setB:
            continue
        jidx = nodeIdx[j]
        nodeRnk[jidx]['deg'] += 1

    # sort nodeRnk
    sortNodeRnk(nodeRnk)
    for k, u in enumerate(nodeRnk):
        nodeIdx[u['node']] = k


def initX(G, lstB, t=0.5):
    # x[lstB[0]] is set to [0,0,0] (anchor)
    x = zeros((len(G.nodes), 3), dtype=float)
    M = zeros((3, 3), dtype=float)
    # TODO: It could be better to select a node with the best data (tight boundaries)
    # create the approximated squadred distance matrix
    D = zeros((4, 4), dtype=float)
    n = len(lstB)
    for iIdx in range(n):
        i = lstB[iIdx]
        for jIdx in range(iIdx + 1, n):
            j = lstB[jIdx]
            lij, uij = G[i][j]['l'], G[i][j]['u']
            dij = ((1 - t) * lij + t * uij) ** 2
            D[iIdx, jIdx], D[jIdx, iIdx] = dij, dij
    # set M
    for i in range(3):
        for j in range(i, 3):
            mij = (D[0, i + 1] + D[0, j + 1] - D[i + 1, j + 1]) / 2.0
            M[i, j], M[j, i] = mij, mij
    U, S, _ = svd(M)
    # scaling cols by the largest singular values
    for i in range(3):
        U[:, i] *= sqrt(S[i])
    # set values
    for i in range(3):
        x[lstB[i + 1]] = U[i]
    return x


def sortNodeRnk(nodeRnk):
    # TODO Some specialized algorithm could be used here
    nodeRnk.sort(key=lambda u: (-u['deg'], u['maxRelRes']))


def initNodeRnk(G, lstB, setB, x):
    # TODO Consider to store maxRelRes of each node on setB to reduce calcs
    nodeRnk = []
    for i in G:  # this loop is O(nnodes * max(deg(G)))
        # ensure that fixed nodes (on B) will be the first ones after sorting
        iNeighsB = [j for j in G.neighbors(i) if setB[j]]
        deg = np.inf if setB[i] else len(iNeighsB)
        maxRelRes = calcNodeMaxRelRes(G, lstB, setB, x, i)
        nodeRnk.append({'node': i, 'deg': deg, 'maxRelRes': maxRelRes, })
    sortNodeRnk(nodeRnk)
    nodeIdx = np.zeros(len(nodeRnk), dtype=int)
    for k in range(len(nodeRnk)):
        nodeIdx[nodeRnk[k]['node']] = k
    return nodeRnk, nodeIdx


def initB(G):
    # TODO We may consider all cliques and get the "best" of them (resp. to some stability criterion) or just a good enough clique for the sake of time efficiency.
    # Reference:
    # Östergård, Patric RJ. "A fast algorithm for the maximum clique problem." Discrete Applied Mathematics 120.1-3 (2002): 197-207.

    # look for a base that allows to fix the maximum number of nodes
    numBases = 0  # number of tested bases
    numNodes = len(G.nodes)
    maxB = []
    maxLenB = 0  # largest base len
    for v1 in G.nodes:
        N1 = set(G.neighbors(v1))
        if len(N1) < 3:
            continue
        for v2 in N1:
            if v2 < v1:  # removing symmetry
                continue
            N2 = set(G.neighbors(v2)) & N1
            if len(N2) < 2:
                continue
            for v3 in N2:
                if v3 < v1 or v3 < v2:  # removing symmetry
                    continue
                N3 = set(G.neighbors(v3)) & N2
                if len(N1) < 1:
                    continue
                for v4 in N3:
                    if v4 < v1 or v4 < v2 or v4 < v3:  # removing symmetry
                        continue
                    numBases += 1
                    # numFixNeighs[i]: num of neighs of vertice i already fixed
                    numFixNeighs = zeros(numNodes, dtype=int)
                    # the base is fixed by construction
                    numFixNeighs[v1], numFixNeighs[v2], numFixNeighs[v3], numFixNeighs[v4] = 4, 4, 4, 4
                    F = {v1, v2, v3, v4}  # nodes to be fixed
                    B = set()  # set of fixed nodes
                    lenB = len(B)
                    while len(F) > 0:
                        v = F.pop()
                        B.add(v)
                        lenB += 1
                        for u in G.neighbors(v):
                            # ensure that u will be added once
                            # u has one more fixed neigh (which is v)
                            numFixNeighs[u] += 1
                            if numFixNeighs[u] == 4:
                                F.add(u)
                    if lenB > maxLenB:
                        maxLenB = lenB
                        maxB = B.copy()
                        lstB = [v1, v2, v3, v4]
                        print('Updating Bmax (Blen = %d)' % lenB)
    print('Optimal base %s found after %d tests.' % (str(lstB), numBases))
    if len(maxB) < numNodes:
        F = set(G.nodes) - maxB
        print('Warning:')
        print('   Only %d/%d nodes can be solved (fixed).' %
                     (maxLenB, numNodes))
        print(
            '   Nodes %s will not fixed (not enough constraints).' % (str([i + 1 for i in F])))
    setB = np.zeros(numNodes, dtype=bool)
    setB[lstB] = True
    return lstB, setB


def solveLSTSQ(G, setB, x, i, t=0.5):
    iNeighB = [j for j in G.neighbors(i) if setB[j]]
    n = len(iNeighB)
    A = np.zeros((n - 1, 3), dtype=float)
    b = np.zeros(n - 1, dtype=float)
    for k, j in enumerate(iNeighB):
        xj = x[j]
        gij = G[i][j]
        lij, uij = gij['l'], gij['u']
        dij = (1.0 - t) * lij + t * uij
        aij = dij ** 2 - norm(xj) ** 2
        if k == 0:
            x0, ai0 = x[j], aij
        else:
            A[k - 1, :] = x0 - xj
            b[k - 1] = (aij - ai0) / 2.0
    print(A)
    print(b)
    bla = lstsq(A, b, rcond=None)[0]
    print(bla)
    if np.mean(bla) > 1e3 or np.mean(bla) < -1e3:
        bla = np.zeros(3)
    x[i] = bla


def solveNLP(G, setB, lstB, x, check, tm, tau=1e-5, lam=0.5):
    # TODO A C/C++ call would have a dramatically impact here (see https://realpython.com/python-bindings-overview/).
    # marshalling data
    L, U = [], []
    locIdxB = {lstB[k]: k for k in range(len(lstB))}
    for i in lstB:
        iloc = locIdxB[i]
        for j in G.neighbors(i):
            if j < i and setB[j]:
                jloc = locIdxB[j]
                gij = G[i][j]
                L.append([iloc, jloc, gij['l']])
                U.append([iloc, jloc, gij['u']])
    L, U = np.array(L), np.array(U)
    # convert list of 3D points to a single array
    y = x[lstB]
    # y /= np.max(y)
    y = y.reshape(3 * len(lstB))
    if check:
        checkDiff(funNLP, y, args=(tau, lam, L, U))
    ans = minimize(funNLP, y, args=(tau, lam, L, U), method='BFGS', jac=True)
    # retrieve solution
    for k, i in enumerate(lstB):
        x[i] = ans.x[(3 * k):(3 * (k + 1))]
